This notebook presents a step by step way to get text embeddings for each patient's report based on SCP codes and confidence scores provided in the PTB-XL dataset:

- Input: 
    - ptbxl_database.csv
    - scp_statements.csv
    - Model from emilyalsentzer/Bio_ClinicalBERT

- Output: 
    - A dictionary with patient ID and text embedding (dimension is 768)
    - Saved to patient_embedding_dict_summed_SCP_structured_w_confidence.pkl


In [ ]:
import os
import numpy as np
import pickle 
import torch
import pandas as pd
import ast
from transformers import AutoTokenizer, AutoModel
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to(device)
model.eval()



In [ ]:
ptbxl_path = "../data/ptbxl/ptbxl_database.csv"
scp_statements_path = "../data/ptbxl/scp_statements.csv"
assert os.path.exists(ptbxl_path) and os.path.exists(scp_statements_path), "ptbxl_database.csv and scp_statements.csv must be downloaded from https://physionet.org/content/ptb-xl/1.0.1/ and placed in the data/ptbxl folder"

report_df = pd.read_csv(ptbxl_path)
report_df.head(5)

def convert_string_to_dict(string):
    return ast.literal_eval(string)
report_df['scp_codes_dict'] = report_df['scp_codes'].apply(convert_string_to_dict)
print ("len of patient",len(report_df))
report_df.head(5)

## iterate over the rows to remove the scp codes with 0

In [ ]:
for index, row in report_df.iterrows():
    remove_keys = []
    for key in row['scp_codes_dict'].keys():
        if row['scp_codes_dict'][key]==0:
            remove_keys.append(key)
    for key in remove_keys:
        row['scp_codes_dict'].pop(key)

report_df.tail(5)

In [ ]:
df = pd.read_csv(scp_statements_path)
print(len(df))
print("SCP code description")
df.head(20)

## for each record, construct its text embeddings. 

In [ ]:
from collections import OrderedDict
text_features = []
patient_embedding_dict = {}
## give an ordered dictionary to cache the SCP embeddings
cache_dict = OrderedDict()

for index, row in report_df.iterrows():
    # for each patient, get the SCP codes and their confidence scores
    sentence_count =0
    encoded_inputs_embedding_list = []
    uncertainty_list = []
    total_prompt = ''
    for key, uncertainty in row['scp_codes_dict'].items():
        sentence_count+=1
        scaled_uncertainty = uncertainty/100.0
        uncertainty_list.append(scaled_uncertainty)
        ## promp is defined here as the SCP code categorty and the SCP code description
        string_text = "[CLS] "+df[df['key']==key]['Statement Category'].values[0]+':'
        string_text+=df[df['key']==key]['SCP-ECG Statement Description'].values[0]+" [SCP]"
        print(string_text)
      
        if key in cache_dict.keys():
            ## if related SCP embedding is already calculated, directly use it
            embeddings = cache_dict[key]
            encoded_inputs_embedding_list.append(embeddings)
        else:
            encoded_inputs = tokenizer(string_text, add_special_tokens=False, truncation = True,return_tensors="pt",max_length=100, padding = 'max_length')
            input_ids = encoded_inputs['input_ids']
            segments_tensors = encoded_inputs['token_type_ids']
            attention_mask = encoded_inputs['attention_mask']
            with torch.inference_mode():
                model_output = model(input_ids = input_ids.to(device),attention_mask= attention_mask.to(device))
                embeddings = model_output.last_hidden_state
                embeddings = torch.mean(embeddings,keepdim=True,dim=1)
                embeddings = embeddings.squeeze(0)
            cache_dict.update({key:embeddings})
            encoded_inputs_embedding_list.append(embeddings)
    multi_embedding =None
    sum_uncertainty = sum(uncertainty_list)
    i= 0
    ## weighted average of SCP embeddings based on confidence scores
    for embedding,uncertainty in zip(encoded_inputs_embedding_list,uncertainty_list):
        if i==0:
            multi_embedding = embedding*(uncertainty/sum_uncertainty)
        else:multi_embedding+=embedding*(uncertainty/sum_uncertainty)
        i+=1
    print(multi_embedding.shape) 
    ## save the dictionary with patient ecg id as key,  and its text embedding as value
    patient_embedding_dict.update({row['ecg_id']:multi_embedding.cpu().numpy().squeeze()})

with open('patient_embedding_dict_summed_SCP_structured_w_confidence.pkl', 'wb') as f:
    pickle.dump(patient_embedding_dict, f)




## visualize different text embeddings with different scp codes with UMAP


In [ ]:
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from umap import UMAP
import seaborn as sns
import plotly.express as px

sns.set(style='white', context='poster', rc={'figure.figsize':(14,10)})


## flatten the dictionary to get the embeddings
input_batch_embeddings = torch.stack([torch.tensor(embedding).detach() for embedding in cache_dict.values()])
feature_df  = pd.DataFrame()
scp_codes = [key for key in cache_dict.keys()]
print((scp_codes))
feature_df = pd.DataFrame(scp_codes)
# feature_df['scp_codes'] = scp_codes
diagnostic_class = [str(df[df['key']==key]['diagnostic_class'].values[0]) for key in cache_dict.keys()]
feature_df['diagnostic_class'] = diagnostic_class
print((diagnostic_class))
print(len(diagnostic_class))
scp_statements = [df[df['key']==key]['SCP-ECG Statement Description'].values[0] for key in cache_dict.keys()]
statement_descriptions = [df[df['key']==key]['SCP-ECG Statement Description'].values[0] for key in cache_dict.keys()]

#

In [ ]:


# proj_2d = umap_2d.fit_transform(input_batch_embeddings.detach().cpu().numpy().squeeze())

sns.set(style='white', context='poster', rc={'figure.figsize':(14,10)})
umap_2d = UMAP(random_state=42)
fig_2d = plt.figure(figsize=(12, 12))
proj_2d = umap_2d.fit_transform(input_batch_embeddings[:,0,:].detach().cpu().numpy())
# df['feature'] = proj_2d.tolist()
feature_df['feature'] = proj_2d.tolist()
fig_2d = px.scatter(proj_2d,x=0,y=1,color = feature_df['diagnostic_class'], 
                    symbol = scp_codes, height=600,width=600)  # O)
fig_2d.show()